In [1]:
%tensorflow_version 1.x
import tensorflow.keras as K
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

TensorFlow 1.x selected.


In [2]:
def preprocess_data(X, Y):
    X = K.applications.resnet50.preprocess_input(X)
    Y = K.utils.to_categorical(Y, 10)

    return X, Y

In [3]:
if __name__ == '__main__':
    # Divide the data in Train and Test Datasets
    (x_train, y_train),(x_test, y_test) = K.datasets.cifar10.load_data()

    x_train, y_train = preprocess_data(x_train, y_train)
    x_test, y_test = preprocess_data(x_test, y_test)

    target_size = (224, 224)
    
    model = K.applications.ResNet50(include_top=False,
                                 weights='imagenet',
                                 input_shape=(224, 224, 3))

    model_1= K.Sequential()
    model_1.add(K.layers.Lambda(lambda image: tf.image.resize(image, target_size))) 
    model_1.add(model)
    model_1.add(K.layers.AveragePooling2D(pool_size=7)) 
    model_1.add(K.layers.Flatten())
    model_1.add(K.layers.Dense(10,activation=('softmax')))

    checkpoint = K.callbacks.ModelCheckpoint(filepath='cifar10.h5',
                                             monitor='val_acc',
                                             mode='max',
                                             verbose=1,
                                             save_best_only=True)
    
    earlys = K.callbacks.EarlyStopping(monitor='val_acc',
                                   mode='max',
                                   verbose=1,
                                   patience=10)

    model_1.compile(optimizer=K.optimizers.RMSprop(learning_rate=1e-4),
                  loss='categorical_crossentropy',
                  metrics=['acc'])

    model_1.fit(x_train, y_train,
                validation_data=(x_test, y_test),
                batch_size=32,
                epochs=10,
                verbose=1,
                callbacks=[checkpoint, earlys])

    model_1.save('cifar10.h5')

170500096/170498071 [==============================] - 6s 0us/step
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
94773248/94765736 [==============================] - 4s 0us/step
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
49984/50000 [============================>.] - ETA: 0s - loss: 0.3439 - acc: 0.8869
Epoch 00001: val_acc improved from -inf to 0.89310, saving model to cifar10.h5
50000/50000 [==============================] - 851s 17ms/sample - loss: 0.3439 - acc: 0.8869 - val_loss: 0.4463 - val_acc: 0.8931
Epoch 2/10
49984/50000 [============================>.] - ETA: 0s - loss: 0.1601 - acc: 0.9482
Epoch 00002: val_acc improved from 0.89310 to 0.92360, saving model to cifar10.h5
50000/50000 [==============================] - 837s 17ms/sample - loss: 0.1601 - acc: 0.9483 - val_loss: 0.2978 - val_acc: 0.9236
Epoch 3/10
49984/50000 [============================>.] - ETA: 0s - loss: 0.1020 - acc: 0.9663
Epoch 00003: val_acc did not im

In [5]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              multiple                  0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
average_pooling2d (AveragePo multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________
